In [2]:
import cv2
import numpy as np

## Exercicio 1

#### Considere a imagem “corruptedRect.png” (em anexo a estes exercícios): 

In [3]:
# https://docs.opencv.org/4.x/d9/d61/tutorial_py_morphological_ops.html

# Carregar a imagem
I = cv2.imread('images/corruptedRect.png', cv2.IMREAD_GRAYSCALE)

# Verificar se a imagem foi carregada corretamente
if I is None:
    print("Erro ao carregar a imagem.")
else:
    # Mostrar a imagem
    cv2.imshow('Imagem', I)
    cv2.waitKey(0)
    cv2.destroyAllWindows() 

Utilizando  uma  sequência  de  operações  morfológicas  (possivelmente  com  elementos estruturantes  diferentes),  tente  obter  a  imagem  de  um  retângulo  branco  tão  perfeito quanto  o  possível  contra  um  fundo  totalmente  negro  –  o retângulo “perfeito” deverá ter uma área idêntica à da figura original. 

In [4]:
# Verificar se a imagem foi carregada corretamente
if I is None:
    print("Erro ao carregar a imagem.")
else:
    # Binarizar a imagem (preto e branco)
    _, binary_I = cv2.threshold(I, 127, 255, cv2.THRESH_BINARY)

    # Definir um kernel retangular (elemento estruturante)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (15, 15))

    # Aplicar Erosão
    erosion = cv2.erode(I,kernel,iterations = 1)

    # Aplicar fechamento para preencher buracos internos
    closed_I = cv2.morphologyEx(erosion, cv2.MORPH_CLOSE, kernel, iterations=3)

    # Aplicar abertura para remover ruídos externos
    opened_I = cv2.morphologyEx(closed_I, cv2.MORPH_OPEN, kernel, iterations=1)

    # Mostrar as imagens
    cv2.imshow('Original_I', I)
    cv2.imshow('opened_I', opened_I)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

## Exercicio 2

Mostre experimentalmente, testando com  uma imagem $I$ e com dois filtros $𝐹1$ e  $𝐹2$ à sua escolha, ambos com a mesma dimensão, que a seguinte igualdade é válida: 


$ 𝐼∗(𝐹1 +𝐹2)=(𝐼∗𝐹2)+(𝐼∗𝐹1)$ 

Notas:
- Poderão existir pequenas diferenças devido a erros de arredondamento nos cálculos realizados; 
- Dependendo dos filtros utilizados, poderá ser necessário realizar as operações em matrizes **float32** em vez de **uint8**, para se poderem tratar resultados que saiam fora da gama [0, 255]. 

In [5]:
# Passo 1: Carregar a imagem (ou criar uma imagem de teste)

# Carregar a imagem
I = cv2.imread('images/spider-man.jpg')

# Verificar se a imagem foi carregada corretamente
if I is None:
    print("Erro ao carregar a imagem.")
else:
    # Redimensionar a imagem para 50% do tamanho original
    scale_percent = 20  # percentagem do tamanho original
    width = int(I.shape[1] * scale_percent / 100)
    height = int(I.shape[0] * scale_percent / 100)
    dim = (width, height)

    I = cv2.resize(I, dim, interpolation=cv2.INTER_AREA)

    # Mostrar a imagem
    cv2.imshow('Imagem', I)
    cv2.waitKey(0)
    cv2.destroyAllWindows() 

In [6]:
# Passo 2: Definir dois filtros (kernels) F1 e F2
F1 = np.array([[1, 0, -1],
               [1, 0, -1],
               [1, 0, -1]], dtype=np.float32)  # Filtro de detecção de bordas verticais

F2 = np.array([[1, 1, 1],
               [0, 0, 0],
               [-1, -1, -1]], dtype=np.float32)  # Filtro de detecção de bordas horizontais

# Passo 3: Calcular F1 + F2
F1_plus_F2 = F1 + F2
print("F1 + F2: \n" + str(F1_plus_F2))

# Passo 4: Aplicar convolução de F1 + F2 na imagem
I_convolved_with_F1_plus_F2 = cv2.filter2D(I, -1, F1_plus_F2, borderType=cv2.BORDER_DEFAULT)

# Normalizar para evitar estouro de valores e converter para uint8
I_convolved_with_F1_plus_F2 = np.clip(I_convolved_with_F1_plus_F2, 0, 255).astype(np.uint8)

# Mostrar a imagem
cv2.imshow('I * (F1 + F2)', I_convolved_with_F1_plus_F2)
cv2.waitKey(0)
cv2.destroyAllWindows()

F1 + F2: 
[[ 2.  1.  0.]
 [ 1.  0. -1.]
 [ 0. -1. -2.]]


In [7]:
# Passo 5: Aplicar convolução de I com F1 e I com F2, e somar os resultados
I_convolved_with_F1 = cv2.filter2D(I, -1, F1, borderType=cv2.BORDER_DEFAULT)
I_convolved_with_F2 = cv2.filter2D(I, -1, F2, borderType=cv2.BORDER_DEFAULT)

# Somar convoluções corretamente (evitando estouro de valores)
sum_of_convolutions = cv2.addWeighted(I_convolved_with_F1, 1, I_convolved_with_F2, 1, 0)

# Normalizar e converter para uint8
sum_of_convolutions = np.clip(sum_of_convolutions, 0, 255).astype(np.uint8)

# Redimensionar imagens para garantir que têm as mesmas dimensões (se necessário)
h, w = I.shape[:2]
h, w = int(h * 0.55), int(w * 0.55)
I_convolved_with_F1 = cv2.resize(I_convolved_with_F1, (w, h))
I_convolved_with_F2 = cv2.resize(I_convolved_with_F2, (w, h))
sum_of_convolutions_copy = cv2.resize(sum_of_convolutions.copy(), (w, h))

# Concatenar horizontalmente
combined = cv2.hconcat([I_convolved_with_F1, I_convolved_with_F2, sum_of_convolutions_copy])

# Mostrar as imagens juntas
cv2.imshow('I * F1 <> I * F2 <> (I * F1) + (I * F2)', combined)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [8]:
# Passo 6: Calcular a diferença entre I * (F1 + F2) e (I * F1) + (I * F2)
difference = cv2.absdiff(I_convolved_with_F1_plus_F2, sum_of_convolutions)
cv2.imshow('Diferença', difference)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [9]:
# Passo 7: Comparar os resultados
# Função para adicionar rótulo à imagem
def add_label(img, label):
    fonte = cv2.FONT_HERSHEY_SIMPLEX
    # Converter para BGR caso a imagem seja em escala de cinza
    if len(img.shape) == 2:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    return cv2.putText(img, label, (10, 30), fonte, 1, (255, 255, 255), 2, cv2.LINE_AA)

# Adicionar rótulos às imagens desejadas
I_labeled = add_label(I.copy(), 'Original')
I_conv_F1_plus_F2_labeled = add_label(I_convolved_with_F1_plus_F2.copy(), 'I * (F1+F2)')
sum_conv_labeled = add_label(sum_of_convolutions.copy(), '(I * F1) + (I * F2)')
diff_labeled = add_label(difference.copy(), 'Diferenca')

# Função para redimensionar a imagem para uma percentagem do seu tamanho original
def resize_image(img, scale_percent):
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    dim = (width, height)
    return cv2.resize(img, dim, interpolation=cv2.INTER_AREA)

# Reduzir todas as imagens para 50% do tamanho original (podes ajustar a percentagem)
I_labeled = resize_image(I_labeled, 90)
I_conv_F1_plus_F2_labeled = resize_image(I_conv_F1_plus_F2_labeled, 90)
sum_conv_labeled = resize_image(sum_conv_labeled, 90)
diff_labeled = resize_image(diff_labeled, 90)

# Organizar as imagens numa matriz 2x2
top_row = cv2.hconcat([sum_conv_labeled, I_conv_F1_plus_F2_labeled])
bottom_row = cv2.hconcat([I_labeled, diff_labeled])
final_image = cv2.vconcat([top_row, bottom_row])

# Exibir a imagem final
cv2.imshow('Resultados', final_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Exercício 3 (convolução, gradiente) 

Desenvolva um programa que calcula e mostra uma estimativa da magnitude do gradiente de uma imagem. Recorde que pode obter as componentes x e y do gradiente usando filtros de Sobel e que a magnitude do gradiente pode ser obtida calculando a norma  euclidiana (L2) entre as duas componentes.  
Teste nas imagens “lastsupper.jpg” e “clocks.jpg”  fornecidas  em  anexo.  Tenha  cuidado com  os  formatos  numéricos  utilizados,  pois  a  aplicação  dos  filtros  de  Sobel  produz resultados fora de [0; 255]. 

In [ ]:
def calcular_gradiente(imagem_path):
    # Carregar a imagem em escala de cinza
    I = cv2.imread(imagem_path, cv2.IMREAD_GRAYSCALE)
    if I is None:
        print("Erro ao carregar a imagem.")
        return
    
    # Aplicar os filtros de Sobel para calcular as derivadas parciais
    Gx = cv2.Sobel(I, cv2.CV_64F, 1, 0, ksize=3)  # Gradiente na direção x
    Gy = cv2.Sobel(I, cv2.CV_64F, 0, 1, ksize=3)  # Gradiente na direção y
    
    # Calcular a magnitude do gradiente
    magnitude = np.sqrt(Gx**2 + Gy**2)
    
    # Normalizar para o intervalo [0, 255] para exibição correta
    magnitude = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
    magnitude = np.uint8(magnitude)
    
    # Mostrar a imagem original e a magnitude do gradiente
    cv2.imshow('Imagem Original', I)
    cv2.imshow('Magnitude do Gradiente', magnitude)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Testar nas imagens fornecidas
calcular_gradiente('images/lastsupper.jpg')
# calcular_gradiente('images/clocks.jpg')
